<a href="https://colab.research.google.com/github/theronitdey/molgenfinalproject2025/blob/main/MolGenFinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Molecular Genetics Final Project**

For my final project, I want to explore how deep learning models, traditionally thought to require massive datasets, can be adapted to small biological datasets through techniques such as data augmentation, transfer learning, and regularization. I want to tailor this towards creating a modelor distinguishing between acute myeloid leukemia (AML) and acute lymphoblastic leukemia (ALL) using supervised learning techniques. I will first expand the effective size of the training set through biologically-informed data augmentation and pretraining strategies, and then develop a compact deep learning model (e.g., a small dense network or a Siamese network) optimized for generalization with minimal data. Rather than focusing on trying to maximize classification accuracy by using a very computationally intensive and complicated model architecture, I want to focus on computational efficiency and interpretability, since that is critical for practical applications in molecular genetics. On a personal level,  building computationally efficient and interpretable models is something I am passionate about and have experience with.

# Part 1: Setting up Imports and Reading in Data

In [1]:
#Various imports necessary to perform computations, develop supervised ML models, and create visualiztaions
import os
import numpy as np
import shutil
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix

from google.colab import drive
drive.mount('/content/drive') #Mounting to Google Drive

Mounted at /content/drive
